In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import random
import sys

In [2]:
path = tf.keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path, encoding='utf-8').read().lower()
print(f"Corpus length: {len(text)}")

600901/600901 [==============================] - 2s 4us/step
Corpus length: 600893


In [3]:
# Create character vocabulary
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [4]:
# Create sequences
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print(f"Number of sequences: {len(sentences)}")

Number of sequences: 200285


In [5]:
# One-hot encode
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool_)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool_)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [6]:
model = Sequential([
    LSTM(128, input_shape=(maxlen, len(chars))),
    Dense(len(chars), activation='softmax')
])

optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

checkpoint = ModelCheckpoint('char_lm_model.h5', monitor='loss', save_best_only=True)
model.fit(X, y, batch_size=128, epochs=20, callbacks=[checkpoint])

Epoch 1/20
1565/1565 [==============================] - 361s 216ms/step - loss: 2.5763
Epoch 2/20
1565/1565 [==============================] - 310s 198ms/step - loss: 2.2158
Epoch 3/20
1565/1565 [==============================] - 224s 143ms/step - loss: 2.0831
Epoch 4/20
1565/1565 [==============================] - 313s 200ms/step - loss: 1.9950
Epoch 5/20
1565/1565 [==============================] - 360s 230ms/step - loss: 1.9246
Epoch 6/20
1565/1565 [==============================] - 264s 169ms/step - loss: 1.8693
Epoch 7/20
1565/1565 [==============================] - 235s 150ms/step - loss: 1.8192
Epoch 8/20
1565/1565 [==============================] - 314s 201ms/step - loss: 1.7772
Epoch 9/20
1565/1565 [==============================] - 282s 180ms/step - loss: 1.7401
Epoch 10/20
1565/1565 [==============================] - 246s 157ms/step - loss: 1.7080
Epoch 11/20
1565/1565 [==============================] - 335s 214ms/step - loss: 1.6793
Epoch 12/20
1565/1565 [==================

In [7]:
model = load_model('char_lm_model.h5')

In [8]:
# Function to sample the next character with temperature
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [9]:
# Generate text
seed_text = text[1000:1040]
generated = seed_text
print(f"\nGenerating with seed: '{seed_text}'")

for temperature in [0.2, 0.5, 1.0]:
    print(f"\n--- Temperature: {temperature}")
    sys.stdout.write(generated)

    for i in range(400):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(seed_text):
            sampled[0, t, char_indices[char]] = 1.

        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = indices_char[next_index]

        generated += next_char
        seed_text = seed_text[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


Generating with seed: 'lute philosophical edifices as the dogma'

--- Temperature: 0.2
lute philosophical edifices as the dogmative of the prost and the spirit the still the same a still to the strength of the strength of the spirit to the prost of the stands the prost still to the strength and the conscious and the strengle the fact the consciously and the still to the conscience of the strong and the sainted to the desire the most strength of the same and such a sortle the still to the actions and the called to the the 

--- Temperature: 0.5
lute philosophical edifices as the dogmative of the prost and the spirit the still the same a still to the strength of the strength of the spirit to the prost of the stands the prost still to the strength and the conscious and the strengle the fact the consciously and the still to the conscience of the strong and the sainted to the desire the most strength of the same and such a sortle the still to the actions and the called to the the may men a